### INTERFACING B/W PANDAS AND MODEL CODE

In [61]:
# to turn a DF into a NumPy array, use the .values property
import pandas as pd
import numpy as np

In [62]:
data = pd.DataFrame({'x0': [1, 2, 3, 4, 5],
                     'x1': [0.01, -0.01, 0.25, -4.1, 0.],
                     'y': [-1.5, 0., 3.6, 1.3, -2.]})
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [63]:
data.columns

Index(['x0', 'x1', 'y'], dtype='object')

In [64]:
data.values

array([[ 1.  ,  0.01, -1.5 ],
       [ 2.  , -0.01,  0.  ],
       [ 3.  ,  0.25,  3.6 ],
       [ 4.  , -4.1 ,  1.3 ],
       [ 5.  ,  0.  , -2.  ]])

In [65]:
# convert back to DF, pass a 2D ndarray with optional column names
df2 = pd.DataFrame(data.values, columns=['one', 'two', 'three'])
df2

,one,two,three
0,1.0,0.01,-1.5
1,2.0,-0.01,0.0
2,3.0,0.25,3.6
3,4.0,-4.10,1.3
4,5.0,0.00,-2.0


In [66]:
# for some models, we may require to use a subset of the columns
# use loc indexing with values
model_cols = ['x0', 'x1']

In [67]:
data.loc[:, model_cols].values

array([[ 1.  ,  0.01],
       [ 2.  , -0.01],
       [ 3.  ,  0.25],
       [ 4.  , -4.1 ],
       [ 5.  ,  0.  ]])

In [68]:
# most of the libraries have native support with pandas
# metadata management may need manual work

In [69]:
data['category'] = pd.Categorical(['a', 'b', 'a', 'a', 'b'],
                                  categories=['a', 'b'])
data

,x0,x1,y,category
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,a
3,4,-4.10,1.3,a
4,5,0.00,-2.0,b


In [70]:
# to replace 'category' column with dummy variables, create dummy variables
# then drop the category column, and join the result
dummies = pd.get_dummies(data.category, prefix='category')
dummies

,category_a,category_b
0,1,0
1,0,1
2,1,0
3,1,0
4,0,1


In [71]:
data_with_dummies = data.drop('category', axis=1).join(dummies)
data_with_dummies

,x0,x1,y,category_a,category_b
0,1,0.01,-1.5,1,0
1,2,-0.01,0.0,0,1
2,3,0.25,3.6,1,0
3,4,-4.10,1.3,1,0
4,5,0.00,-2.0,0,1


### CREATING MODEL DESCRIPTIONS WITH PATSY

In [72]:
# describes statistical models with small string-based 'formula-syntax'
# well supported for specifying linear models in statsmodel
# Patsy has special string syntax formulas: y ~ x0 + x1
# the above formula does not mean add but are terms in the design matrix
# patsy.dmatrices takes a DF/dict array and produces design matrices for a linear model

In [73]:
data = pd.DataFrame({
    'x0': [1, 2, 3, 4, 5],
    'x1': [0.01, -0.01, 0.25, -4.1, 0.],
    'y': [-1.5, 0., 3.6, 1.3, -2.]
})

data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [74]:
import patsy

In [75]:
y, X = patsy.dmatrices('y ~ x0 + x1', data)

In [76]:
y

DesignMatrix with shape (5, 1)
     y
  -1.5
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

In [77]:
X

DesignMatrix with shape (5, 3)
  Intercept  x0     x1
          1   1   0.01
          1   2  -0.01
          1   3   0.25
          1   4  -4.10
          1   5   0.00
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'x1' (column 2)

In [78]:
np.asarray(y)

array([[-1.5],
       [ 0. ],
       [ 3.6],
       [ 1.3],
       [-2. ]])

In [79]:
np.asarray(X)

array([[ 1.  ,  1.  ,  0.01],
       [ 1.  ,  2.  , -0.01],
       [ 1.  ,  3.  ,  0.25],
       [ 1.  ,  4.  , -4.1 ],
       [ 1.  ,  5.  ,  0.  ]])

In [80]:
# intercept term is a convention for linear models like ordinary least squares regression
# suppress it by adding '+ 0' to the model
patsy.dmatrices('y ~ x0 + x1 + 0', data)[1]

DesignMatrix with shape (5, 2)
  x0     x1
   1   0.01
   2  -0.01
   3   0.25
   4  -4.10
   5   0.00
  Terms:
    'x0' (column 0)
    'x1' (column 1)

In [81]:
# patsy objs can be passed directly into algorithms
# ex. numpy.linalg.lstsq - performs an ordinary least sq. regression
coef, resid, _, _ = np.linalg.lstsq(X, y, rcond=None)

In [82]:
# model metadata is retained in the design_info attribute
# this will reattach the model column names to the fitted coeff. to obtain a Series

In [83]:
coef

array([[ 0.31290976],
       [-0.07910564],
       [-0.26546384]])

In [84]:
coef = pd.Series(coef.squeeze(), index=X.design_info.column_names)
coef

Intercept    0.312910
x0          -0.079106
x1          -0.265464
dtype: float64

### DATA TRANSFORMATIONS IN PATSY FORMULAS

In [85]:
# mix Python code into Patsy formulas
# while evaluating the formula the lib will try to find the functions used in the enclosing scope

In [86]:
y, X = patsy.dmatrices('y ~ x0 + np.log(np.abs(x1) + 1)', data)

In [87]:
X

DesignMatrix with shape (5, 3)
  Intercept  x0  np.log(np.abs(x1) + 1)
          1   1                 0.00995
          1   2                 0.00995
          1   3                 0.22314
          1   4                 1.62924
          1   5                 0.00000
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'np.log(np.abs(x1) + 1)' (column 2)

In [88]:
# standardizing and centering is a common variable transformation
# standardizing (to mean 0 and variance 1)
# centering (subtracting the mean)

In [89]:
y, X = patsy.dmatrices('y ~ standardize(x0) + center(x1)', data)

In [90]:
X

DesignMatrix with shape (5, 3)
  Intercept  standardize(x0)  center(x1)
          1         -1.41421        0.78
          1         -0.70711        0.76
          1          0.00000        1.02
          1          0.70711       -3.33
          1          1.41421        0.77
  Terms:
    'Intercept' (column 0)
    'standardize(x0)' (column 1)
    'center(x1)' (column 2)

In [91]:
# stateful transitions - use statistics like the mean or std of original dataset
# when transforming a new dataset
# patsy.build_design_matrices function can apply transformations to new sample data
# this is performed using the saved info. from the original in-sample dataset

In [92]:
new_data = pd.DataFrame({
    'x0': [6, 7, 8, 9],
    'x1': [3.1, -0.5, 0, 2.3],
    'y': [1, 2, 3, 4]
})

In [93]:
new_X = patsy.build_design_matrices([X.design_info], new_data)
new_X

[DesignMatrix with shape (4, 3)
   Intercept  standardize(x0)  center(x1)
           1          2.12132        3.87
           1          2.82843        0.27
           1          3.53553        0.77
           1          4.24264        3.07
   Terms:
     'Intercept' (column 0)
     'standardize(x0)' (column 1)
     'center(x1)' (column 2)]

In [94]:
# plus '+' does not mean addition in context of Patsy
# add columns from a dataset by name
# wrap them in the special I function
y, X = patsy.dmatrices('y ~ I(x0 + x1)', data)

In [95]:
X

DesignMatrix with shape (5, 2)
  Intercept  I(x0 + x1)
          1        1.01
          1        1.99
          1        3.25
          1       -0.10
          1        5.00
  Terms:
    'Intercept' (column 0)
    'I(x0 + x1)' (column 1)

In [96]:
# categorical data has a special class of transformations

### CATEGORICAL DATA AND PATSY

In [97]:
# non-numerical data can be transformed for a model design matrix
# using non-numerical terms in a Patsy formula will result in conversion to dummy variables by default
# if any intercept occured then the levels will be left out to avoid collinearity

In [98]:
data = pd.DataFrame({
    'key1': ['a', 'a', 'b', 'b', 'a', 'b', 'a', 'b'],
    'key2': [0, 1, 0, 1, 0, 1, 0, 0],
    'v1': [1, 2, 3, 4, 5, 6, 7, 8],
    'v2': [-1, 0, 2.5, -0.5, 4.0, -1.2, 0.2, -1.7]
})

In [99]:
y, X = patsy.dmatrices('v2 ~ key1', data)

In [100]:
X

DesignMatrix with shape (8, 2)
  Intercept  key1[T.b]
          1          0
          1          0
          1          1
          1          1
          1          0
          1          1
          1          0
          1          1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)

In [101]:
# if you omit the intercept from the model
# then columns for each category value will be included in the model design matrix

In [102]:
y, X = patsy.dmatrices('v2 ~ key1 + 0', data)

In [103]:
X

DesignMatrix with shape (8, 2)
  key1[a]  key1[b]
        1        0
        1        0
        0        1
        0        1
        1        0
        0        1
        1        0
        0        1
  Terms:
    'key1' (columns 0:2)

In [104]:
# numeric columns can be intercepted as categorical with the c function
y, X = patsy.dmatrices('v2 ~ C(key2)', data)

In [105]:
X

DesignMatrix with shape (8, 2)
  Intercept  C(key2)[T.1]
          1             0
          1             1
          1             0
          1             1
          1             0
          1             1
          1             0
          1             0
  Terms:
    'Intercept' (column 0)
    'C(key2)' (column 1)

In [106]:
# when using multiple categorical terms in a model, complications arise
# including interaction terms of form key1:key2, used in analysis of variance (ANOVA) models

In [107]:
data

,key1,key2,v1,v2
0,a,0,1,-1.0
1,a,1,2,0.0
2,b,0,3,2.5
3,b,1,4,-0.5
4,a,0,5,4.0
5,b,1,6,-1.2
6,a,0,7,0.2
7,b,0,8,-1.7


In [108]:
data['key2'] = data['key2'].map({
    0: 'zero',
    1: 'one'
})

In [109]:
data

,key1,key2,v1,v2
0,a,zero,1,-1.0
1,a,one,2,0.0
2,b,zero,3,2.5
3,b,one,4,-0.5
4,a,zero,5,4.0
5,b,one,6,-1.2
6,a,zero,7,0.2
7,b,zero,8,-1.7


In [110]:
y, X = patsy.dmatrices('v2 ~ key1 + key2', data)

In [111]:
X

DesignMatrix with shape (8, 3)
  Intercept  key1[T.b]  key2[T.zero]
          1          0             1
          1          0             0
          1          1             1
          1          1             0
          1          0             1
          1          1             0
          1          0             1
          1          1             1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)

In [112]:
y, X = patsy.dmatrices('v2 ~ key1 + key2 + key1:key2', data)

In [113]:
X

DesignMatrix with shape (8, 4)
  Intercept  key1[T.b]  key2[T.zero]  key1[T.b]:key2[T.zero]
          1          0             1                       0
          1          0             0                       0
          1          1             1                       1
          1          1             0                       0
          1          0             1                       0
          1          1             0                       0
          1          0             1                       0
          1          1             1                       1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)
    'key1:key2' (column 3)

### INTRODUCTION TO STATSMODEL